In [7]:
import numpy as np
import pandas as pd
import datetime as dt
from pandas_datareader import data

#Executing Trade Signals for Reliance on NSE for 3 years starting from 15th January 2010

portfolio  = 1500
daysnumber = 1096
stock_list = ['RELIANCE.NS']


def read_data(stock_list, daysnumber):
    df = pd.DataFrame()
    for ticker in stock_list:
        df[ticker] = data.DataReader(ticker, 'yahoo', start = '1/1/2010')['Adj Close']
    return df.head(daysnumber)

stockprices = read_data(stock_list, daysnumber)

nav = pd.DataFrame(index = stockprices.tail(daysnumber - 14).index)
nav = nav.assign(leftover = np.zeros(daysnumber - 14), stock = np.zeros(daysnumber - 14), rsi = np.zeros(daysnumber - 14))
nav.iloc[0,0] = portfolio


def RSI(price_data):
    delta = price_data.diff()
    up = delta.copy()
    down = delta.copy()
    
    up[up < 0] = 0
    down[down > 0] = 0
    roll_up = up.mean()
    roll_down = down.abs().mean()
    RS  = roll_up / roll_down
    RSI = (100 - (100/(1.0 + RS)))[0]
    
    return RSI
    

signal = 0
prev_signal = 0

for index, row in nav.iloc[1:].iterrows():
    signal = np.sign(signal + RSI(stockprices.loc[:index].tail(14)))
    leftover = nav.loc[:index].tail(2).head(1).iloc[0,0]
    
    if (signal == -1):
        nav.loc[index, 'leftover'] = leftover
        nav.loc[index, 'stock'] = 0
        nav.loc[index, 'rsi'] = RSI(stockprices.loc[:index].tail(14))
        continue
    
    if (prev_signal == 0 and signal == 1):
        #BUY
        nav.loc[index, 'leftover'] = leftover - stockprices.loc[index][0]
        nav.loc[index, 'stock'] = stockprices.loc[index][0]
        nav.loc[index, 'rsi'] = RSI(stockprices.loc[:index].tail(14))
    
    if (prev_signal == 1 and signal == 1):
        #HOLD
        nav.loc[index, 'leftover'] = leftover
        nav.loc[index, 'stock'] = stockprices.loc[index][0]
        nav.loc[index, 'rsi'] = RSI(stockprices.loc[:index].tail(14))
        
    if (prev_signal == 1 and signal == 0):
        #SELL
        nav.loc[index, 'leftover'] = leftover + stockprices.loc[index][0]
        nav.loc[index, 'stock'] = 0
        nav.loc[index, 'rsi'] = RSI(stockprices.loc[:index].tail(14))
        
    if (prev_signal == 0 and signal == 0):
        #WAIT
        nav.loc[index, 'leftover'] = leftover 
        nav.loc[index, 'stock'] = 0
        nav.loc[index, 'rsi'] = RSI(stockprices.loc[:index].tail(14))
        
    prev_signal = signal

nav

,leftover,stock,rsi
Date,,,
2010-01-22,1500.000000,0.000000,0.000000
2010-01-25,1055.252869,444.747131,35.857223
2010-01-27,1055.252869,438.558502,25.298949
2010-01-28,1055.252869,442.655823,30.072505
2010-01-29,1055.252869,446.518250,38.464008
...,...,...,...
2014-06-02,1055.252869,508.163208,60.433007
2014-06-03,1055.252869,514.361389,61.225562
2014-06-04,1055.252869,505.956238,49.642490


In [9]:
stockprices

,RELIANCE.NS
Date,
2010-01-04,459.023529
2010-01-05,456.974915
2010-01-06,464.358582
2010-01-07,472.062347
2010-01-08,470.824646
...,...
2014-06-02,508.163208
2014-06-03,514.361389
2014-06-04,505.956238
